In [1]:
import pandas as pd
import numpy as np

from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_absolute_error

### Задание 1

#### Создание тестового набора данных

Данный датасет представляет собой информацию о параметрах жилого помещения и его стоимости. Причём так сложилось, что на стоимость данных жилых помещений оказывают большое влияние 2 параметра: количество комнат и год постройки. Оставшиеся два параметра (площадь и количество гаражей) несущественны.

In [2]:
samples = 100

square = np.random.randint(35, 120, samples)
num_rooms = np.random.randint(1, 5, samples)
num_garage = np.random.randint(0, 2, samples)
built_year = np.random.randint(1, 100, samples)

price =  num_rooms * built_year + num_rooms ** 2

data = pd.DataFrame({"square": square, "num_rooms": num_rooms, "num_garage": num_garage, "built_year": built_year, "price": price})

In [3]:
data.describe()

,square,num_rooms,num_garage,built_year,price
count,100.00000,100.000000,100.000000,100.000000,100.00000
mean,77.55000,2.400000,0.560000,54.140000,133.59000
std,22.73669,0.994937,0.498888,26.253627,86.22839
min,35.00000,1.000000,0.000000,5.000000,12.00000
25%,58.75000,2.000000,0.000000,34.250000,68.00000
50%,78.00000,2.000000,1.000000,52.000000,114.00000
75%,98.00000,3.000000,1.000000,78.000000,174.75000
max,119.00000,4.000000,1.000000,99.000000,412.00000


#### Поиск модели

In [4]:
model = LinearRegression()

Т.к. мы "не знаем" закона, которому подчиняются данные для предсказания цены, то сделаем первое предположение, что цена в какой-то мере зависит от площади и количества комнат.

In [5]:
X = data[["square", "num_rooms"]]
Y = data["price"]

model.fit(X, Y)
predictions = model.predict(X)

print("Точность предсказания: ", round(model.score(X, Y), 4))
print("MAE: ", round(mean_absolute_error(predictions, Y), 4))
print("Веса модели: ", [round(weight, 4) for weight in model.coef_])

Точность предсказания:  0.3247
MAE:  54.3047
Веса модели:  [-0.3233, 47.4161]


Как видно из результатов, наша модель имеет малую точность предсказания (всего 32%). Также параметр МАЕ является не очень удовлетворительным, т.к. при mean/min/max равных 134/12/412 в среднем ошибаться на 54 довольно плохо.

Попробуем, например, заменить параметр количество комнат на год постройки.

In [6]:
X = data[["square", "built_year"]]
Y = data["price"]

model.fit(X, Y)
predictions = model.predict(X)

print("Точность предсказания: ", round(model.score(X, Y), 4))
print("MAE: ", round(mean_absolute_error(predictions, Y), 4))
print("Веса модели: ", [round(weight, 4) for weight in model.coef_])

Точность предсказания:  0.5132
MAE:  47.6112
Веса модели:  [-0.6234, 2.269]


Результаты немного улучшились, но, по-прежнему, являются неудовлетворительными.

Возьмём предельный случай и предположим, что цена зависит от всех признаков.

In [7]:
X = data[["square", "num_rooms", "num_garage", "built_year"]]
Y = data["price"]

model.fit(X, Y)
predictions = model.predict(X)

print("Точность предсказания: ", round(model.score(X, Y), 4))
print("MAE: ", round(mean_absolute_error(predictions, Y), 4))
print("Веса модели: ", [round(weight, 4) for weight in model.coef_])

Точность предсказания:  0.9129
MAE:  18.4053
Веса модели:  [-0.1118, 56.0308, -5.7197, 2.5562]


В этот раз результаты намного лучше. Их в принципе можно считать удовлетворительными, даже без учёта ввода дополнительных параметров. Но можно ли улучшить модель? Попробуем для начала воспользоваться полиномиальными функциями.

In [8]:
pf = PolynomialFeatures(2)

X = pf.fit_transform(data[["square", "num_rooms", "num_garage", "built_year"]])
Y = data["price"]

model.fit(X, Y)

print("Варианты коэффициентов: ", pf.get_feature_names())
print("Значения коэффициентов: ", [round(weight) for weight in model.coef_])

Варианты коэффициентов:  ['1', 'x0', 'x1', 'x2', 'x3', 'x0^2', 'x0 x1', 'x0 x2', 'x0 x3', 'x1^2', 'x1 x2', 'x1 x3', 'x2^2', 'x2 x3', 'x3^2']
Значения коэффициентов:  [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0]


Как видно из расчётов, следующие комбинации коэффициентов: 1 * x1 * x2 и 1 * x2 ** 2 (т.е. 1 * num_rooms * built_year и num_rooms ** 2) - являются существенными для данного датасета. Попробуем добавить в наш датасет два новых параметра: num_rooms_built_year и num_rooms_2.

In [9]:
data["num_rooms_built_year"] = data["num_rooms"] * data["built_year"]
data["num_rooms_2"] = data["num_rooms"] ** 2

Теперь сделаем предположение, что цена зависит только от двух новых параметров.

In [10]:
X = data[["num_rooms_built_year", "num_rooms_2"]]
Y = data["price"]

model.fit(X, Y)
predictions = model.predict(X)

print("Точность предсказания: ", round(model.score(X, Y), 4))
print("MAE: ", round(mean_absolute_error(predictions, Y), 4))
print("Веса модели: ", [round(weight, 4) for weight in model.coef_])

Точность предсказания:  1.0
MAE:  0.0
Веса модели:  [1.0, 1.0]


Как видно из последних расчётов, предсказания полностью совпадают с реальными ценами. Таким образом, если бы мы могли каким-либо образом правильно предположить новые параметры, то получили бы аналогичные результаты без полиномиальных функций.